## The Tensor Network Data Structure Used By PicoQuant

In this notebook we provide some details about how tensor networks are stored in PicoQuant. The data structure that PicoQuant uses to represent a quantum circuit as a tensor network consists of three julia structs: 
1. A Node struct representing a tensor in the tensor network.
2. An Edge struct reprsenting an edge in the tensor network graph.
3. A TensorNetworkCircuit struct holding references to all the nodes and edges in the tensor network.

We will first create a tensor network for a quantum circuit and then briefly describe and illustrate the composition/use of these structs.

In [ ]:
using PicoQuant

# To aid with the illustration we create the same GHZ quantum circuit from the introduction notebook.
qasm_str = """OPENQASM 2.0;
              include "qelib1.inc";
              qreg q[3];
              h q[0];
              cx q[0],q[1];
              cx q[1],q[2];"""

circ = load_qasm_as_circuit(qasm_str)
InteractiveBackend()
tn = convert_qiskit_circ_to_network(circ)
add_input!(tn, "000")

#### TensorNetworkCircuit struct

The TensorNetworkCircuit struct plays the role of a parent struct for the nodes and edges in a network. Namely, it holds references to all the Node and Edge instances representing a particular tensor network. This makes it convenient to access and manipulate nodes/edges. The fields of the TensorNetworkCircuit structure as listed below:

1. An integer for the number of qubits in the circuit being represented.
2. An array of index labels connected to the input qubits.
3. An array of index labels connected to the output qubits.
4. A dictionary containing the nodes of the tensor network.
5. A dictionary containing the edges of the tensor network.
6. A dictionary of counters keeping track of how many nodes and edges are in the tensor network.

When a node or edge is added to the tensor network, the TensorNetworkCircuit instance is given the corresponding Node/Edge struct and a unique label (saved as a julia symbol) to be used as an identifier for the struct. The pair are save in the mentioned dictionary with the unique label as the key and the struct as the value.

The cell below will display the ordered dictionaries containing the network nodes and edges. It also plots a graph of the tensor network for comparison with the details of nodes/edges given below the next cell.

In [ ]:
# We can access the nodes of the network as follows:
nodes = tn.nodes
display(nodes)

# Similarly, we can access the edges of the network as follows:
edges = tn.edges
display(edges)

# We plot a graph of the tensor network with labels indicating the symbols used to identify different nodes/edges.
plot(tn, showlabels=true)

#### Node struct
The main purpose of the Node struct is to hold an array of index labels (saved as julia symbols) corresponding to the indices of the tensor the node represents. These index labels are used to determine how tensors should be reshaped, permuted and contracted whenever the tensor network is altered. The fields of the node struct are:

1. An array of julia symbols representing tensor indices
2. An array of integers, one for each index. These integers are the dimensions of the corresponding tensor indices. (TODO: current branch has symbol labeling the tensor data instead of the index dimensions)

Note, tensor data is not stored in the Node struct or in the TensorNetworkCircuit struct. Instead, tensor data is stored by the backend the user initialises PicoQuant with before any tensors are created. When the user adds a gate to the network, a symbol is created as a label for the gate data. Then both the label and tensor data are passed to the backend for storing. The TensorNetworkCircuit struct also saves the same label for the tensor data along with the corresponding Node struct in a dictionary.

Below, we get the instance of Node corrsponding to the first tensor added to the network and show how we typically access the indices of a node. We can compare the printed indices with those displayed in the above graph of the tensor network and see that they match as expected.

In [ ]:
node = tn.nodes[:node_1]
println("The indices contained in node 1 are: ", node.indices)

#### Edge struct

Edge structs are used to keep track of which nodes are connected by common/contracted indices. When a node is added to the tensor network, an edge struct is created for each index of the node (if one doesn't already exist) and is stored with the corresponding index label in the edges dictionary mentioned at the beginning of this notebook. Each edge struct will be given two gate labels to store that identify the two nodes in the network which share the corrsponding index. If the index associated with an edge is an open index, then one of the gate labels in the edge struct is replaced with the constant 'nothing'.

If none of the gates added to the circuit are decomposed into tensors connected by virtual bonds, the tensor network natrually forms a directed graph with edges originating from the input qubits and moving towards the output qubits. As such, the gate labels stored in an Edge are saved under the variable names src (source) and dst (destination).

In [ ]:
first_edge = tn.edges[:index_1]
println("The source node of the edge labeled :index_1 is: ", first_edge.src)
println("The destination node of the edge labeled :index_1 is: ", first_edge.dst)